In [ ]:
import pyodbc
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Connect to the database
conn_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:finance-datacenter.database.windows.net,1433;Database=high-frequency-data;Uid=CloudSA133cd0dc;Pwd=cloud.admin2023;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

In [ ]:
cnx = pyodbc.connect(conn_string)
cursor = cnx.cursor()

cursor.execute(
    "SELECT DISTINCT(date_time) FROM [dbo].[ive_tickbidask] WHERE date_time >= '2022-01-01 00:00:00.000';"
)


dates = cursor.fetchall()


cursor.close()
cnx.close()


dates = sorted(list(set([date[0].date() for date in dates])))

In [ ]:
import asyncio
from typing import Tuple
from datetime import date


async def get_data_from_date(date: date) -> Tuple[list[float], date]:
    """Get data from a specific date

    Args:
        date (date): date

    Returns:
        Tuple[list[float], date]: list of prices at the specific date
    """
    cnx = pyodbc.connect(conn_string)
    cursor = cnx.cursor()

    cursor.execute(
        "SELECT price FROM [dbo].[ive_tickbidask] WHERE date_time >= '"
        + str(date)
        + " 00:00:00.000' AND date_time <= '"
        + str(date)
        + " 23:59:59.999';"
    )

    data = [price[0] for price in cursor.fetchall()]

    cursor.close()
    cnx.close()

    return data, date


async def get_data(dates: list[dates]):
    """Gather all data

    Args:
        dates (list[dates]): list of dates

    Yields:
        Tuple[list[float], date]: data and date
    """
    for future in asyncio.as_completed([get_data_from_date(date) for date in dates]):
        yield await future

In [ ]:
volatilities = {}
async for data, date in get_data(dates):
    returns = np.diff(data) / data[:-1]
    volatilities[date] = np.sqrt(np.sum(returns**2) / len(returns)) * 100

In [ ]:
volatilities = dict(sorted(volatilities.items(), key=lambda x: x[0]))

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(volatilities.keys(), volatilities.values())
plt.title("Daily Volatility of IVE")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()